<a href="https://colab.research.google.com/github/Prathap200529/spam-sms-detector/blob/main/spam_sms_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# first we connect to drive where the sample sms file is saved

from google.colab import drive
import pandas as pd
import csv

drive.mount('/content/drive')
file_url = '/content/drive/My Drive/spam sms detector/spam.csv'

# Specify the encoding as 'latin1' to handle the problematic characters
data = pd.read_csv(file_url,usecols=[0, 1],encoding='latin1')
data.head()

Mounted at /content/drive


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# now we are removing all the unnecessary texts or special characters
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')

stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

print(stopwords[:5])
print(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['a', 'about', 'above', 'after', 'again']
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [5]:
!pip install nltk
import nltk
import string

# Download the necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the missing 'punkt_tab' data

stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

#processing the sms to long sentence to word by word
def pre_process(v2):
    remove_punct = "".join([word.lower() for word in v2 if word not in punctuation])
    tokenize = nltk.tokenize.word_tokenize(remove_punct)
    remove_stopwords = [word for word in tokenize if word not in stopwords]
    return remove_stopwords

# Assuming 'data' is your DataFrame
# adding a column to our data with our processed messages
data['processed'] = data['v2'].apply(lambda x: pre_process(x))

print(data['processed'].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


0    [go, jurong, point, crazy, available, bugis, n...
1                       [ok, lar, joking, wif, u, oni]
2    [free, entry, 2, wkly, comp, win, fa, cup, fin...
3        [u, dun, say, early, hor, u, c, already, say]
4    [nah, dont, think, goes, usf, lives, around, t...
Name: processed, dtype: object


In [6]:
# differentating ham and spam
def categorize_words():
    spam_words = []
    ham_words = []
    #handling messages associated with spam
    for v2 in data['processed'][data['v1'] == 'spam']:
        for word in v2:
            spam_words.append(word)
    #handling messages associated with ham
    for v2 in data['processed'][data['v1'] == 'ham']:
        for word in v2:
            ham_words.append(word)
    return spam_words, ham_words

spam_words, ham_words = categorize_words()

print(spam_words[:5])
print(ham_words[:5])

['free', 'entry', '2', 'wkly', 'comp']
['go', 'jurong', 'point', 'crazy', 'available']


In [7]:
# iterating all the word from input and counting their occourance
def predict(v2):
    spam_counter = 0
    ham_counter = 0
    #count the occurances of each word in the sms-v2 string
    for word in v2:
        spam_counter += spam_words.count(word)
        ham_counter += ham_words.count(word)
    print('***RESULTS***')
    #if the message is ham
    if ham_counter > spam_counter:
        accuracy = round((ham_counter / (ham_counter + spam_counter) * 100))
        print('messege is not spam, with {}% certainty'.format(accuracy))
    #if the message could be equally spam and ham
    elif ham_counter == spam_counter:
        print('message could be spam')
    #if the message is spam
    else:
        accuracy = round((spam_counter / (ham_counter + spam_counter)* 100))
        print('message is spam, with {}% certainty'.format(accuracy))

In [8]:
user_input = input("Please type a spam or ham message to check if function predicts accurately\n")
#pre-processing the input before prediction
processed_input = pre_process(user_input)

predict(processed_input)

Please type a spam or ham message to check if function predicts accurately
Ok lar... Joking wif u oni...	
***RESULTS***
messege is not spam, with 90% certainty
